In [6]:
from typing import List, Tuple
from itertools import combinations
from math import factorial, exp
from random import randint, sample, uniform

from tqdm import tqdm
import numpy as np

### Constantes

In [7]:
# Os valores estão setados pra 7 apenas para que a resolução por brute force possa executar em tempo razoavel
TAM_TABULEIRO = 7 # Sempre quadrado
N_RAINHAS = 7

### Metodos auxiliares

In [8]:
def dupla_valida(r1:List[int], r2:List[int]) -> bool:
    mesma_linha = r1[1] == r2[1]
    mesma_coluna = r1[0] == r2[0]
    mesma_diagonal_primaria = r1[0]-r1[1] == r2[0]-r2[1]
    mesma_diagonal_secundaria = r1[0]+r1[1] == r2[0]+r2[1]
    return not (mesma_linha or mesma_coluna or mesma_diagonal_primaria or mesma_diagonal_secundaria)

def is_valid(rainhas:List[List[int]]) -> bool:
    for r1, r2 in combinations(rainhas,2):
        if not dupla_valida(r1,r2):
            return False
    return True

def duplicada(rainha:List[int], rainhas:List[List[int]]) -> bool:
    for r in rainhas:
        if rainha[0] == r[0] and rainha[1] == r[1]:
            return False
    return True

def converter_1d_para_2d(indice:int, linhas:int, colunas:int) -> Tuple[int]:
    x = int(indice/colunas)
    y = indice%linhas
    return (x,y)

def score(rainhas:List[List[int]]) -> int:
    score = 0
    for comb in combinations(rainhas, 2):
        score += dupla_valida(*comb)
    return score

def max_score(n_rainhas:int) -> int:
    return len(list(combinations(range(n_rainhas), 2)))

def imprime_tabuleiro(tam_tabuleiro:int, rainhas:List[List[int]]) -> None:
    tabuleiro = [['-' for _ in range(tam_tabuleiro)] for __ in range(tam_tabuleiro)]
    for rainha in rainhas:
        tabuleiro[rainha[0]][rainha[1]] = 'X'
    tabuleiro = '\n'.join(['|'.join(linha) for linha in tabuleiro])
    print(tabuleiro)

def move(rainha:List[int], tam_tabuleiro:int):
    # Move x
    atual_x = rainha[0]
    novo_x = atual_x + randint(-1,1)
    novo_x = novo_x if novo_x > 0 and novo_x < tam_tabuleiro-1 else atual_x
    # Move y
    atual_y = rainha[1]
    novo_y = atual_y + randint(-1,1)
    novo_y = novo_y if novo_y > 0 and novo_y < tam_tabuleiro-1 else atual_y
    # Nova rainha
    return (novo_x, novo_y)

def gera_semelhante(solucao:List[List[int]], tam_tabuleiro:int) -> List[List[int]]:
    nova_solucao = solucao.copy()
    # Sorteia uma quantidade de rainhas para serem modificadas, 
    # de 1 a 3 (ou a quantidade maxima de rainhas caso hajam menos de 3)
    n_rainhas_modificadas = randint(1,max(3,len(solucao)))
    # Sorteia n rainhas (apenas seus indices)
    rainhas_idx = sample(range(len(solucao)),k=n_rainhas_modificadas)
    # Move-as para casas adjacentes
    for rainha_idx in rainhas_idx:
        nova_solucao[rainha_idx] = move(solucao[rainha_idx],tam_tabuleiro)
    return nova_solucao

### Algoritmos para resolução do problema

In [9]:
def brute_force(tamanho_tabuleiro:int, n_rainhas:int) -> List[int]:
    max_combinacoes = factorial(tamanho_tabuleiro**2)/(factorial(tamanho_tabuleiro**2-n_rainhas)*factorial(n_rainhas))
    for comb in tqdm(combinations(range(tamanho_tabuleiro**2), n_rainhas),total=max_combinacoes):
        rainhas = [converter_1d_para_2d(comb_1d,tamanho_tabuleiro,tamanho_tabuleiro) for comb_1d in comb]
        if is_valid(rainhas):
            return rainhas
    return []
            
def subida_encosta(tamanho_tabuleiro:int, n_rainhas:int, n_vizinhos:int=100, max_iter:int=100000) -> List[int]:
    score_max = max_score(n_rainhas)
    # Inicia com uma solucao aleatoria
    solucao = [(randint(0,tamanho_tabuleiro-1), randint(0,tamanho_tabuleiro-1)) for _ in range(n_rainhas)]
    melhor_score = score(solucao)
    i = 0
    while i < max_iter:
        # Gera vizinhos parecidos
        vizinhos = [gera_semelhante(solucao, tamanho_tabuleiro) for _ in range(n_vizinhos)]
        scores = [score(vizinho) for vizinho in vizinhos]
        melhor_vizinho_idx = np.argmax(scores)
        if melhor_score < scores[melhor_vizinho_idx]:
            solucao = vizinhos[melhor_vizinho_idx]
            melhor_score = scores[melhor_vizinho_idx]
            # Verifica se atingiu o score maximo, caso positivo nao temos motivo para continuar
            if melhor_score == score_max:
                i = max_iter
        else:
            break
        i+=1
    return solucao

def simulated_annealing(tamanho_tabuleiro:int, n_rainhas:int, n_vizinhos:int=100, fator:float=0.95):
    score_max = max_score(n_rainhas)
    temp_inicial = 100
    temp_final = 1
    solucao = [(randint(0,tamanho_tabuleiro-1), randint(0,tamanho_tabuleiro-1)) for _ in range(n_rainhas)]
    melhor_score = score(solucao)

    temp_atual = temp_inicial
    while temp_atual > temp_final:
        vizinhos = [gera_semelhante(solucao, tamanho_tabuleiro) for _ in range(n_vizinhos)]
        scores = [score(vizinho) for vizinho in vizinhos]
        melhor_vizinho_idx = np.argmax(scores)
        
        diferenca_score = melhor_score - scores[melhor_vizinho_idx]
        if diferenca_score >= 0:
            solucao = vizinhos[melhor_vizinho_idx]
            melhor_score = scores[melhor_vizinho_idx]
            # Verifica se atingiu o score maximo, caso positivo nao temos motivo para continuar
            if melhor_score == score_max:
                break
        else:
            if uniform(0,1) < exp(diferenca_score/temp_atual):
                solucao = vizinhos[melhor_vizinho_idx]
                melhor_score = scores[melhor_vizinho_idx]
        temp_atual = temp_atual * fator
    return solucao

### Simulação

In [10]:
solucoes = {}


solucoes['subida_encosta'] = subida_encosta(TAM_TABULEIRO, N_RAINHAS, n_vizinhos=100)
imprime_tabuleiro(TAM_TABULEIRO, solucoes['subida_encosta'])
print(f"Posições das rainhas: {solucoes['subida_encosta']}")
print(f"score: {score(solucoes['subida_encosta'])}")

print('='*100)

solucoes['recozimento_simulado'] = simulated_annealing(TAM_TABULEIRO, N_RAINHAS, n_vizinhos=100)
imprime_tabuleiro(TAM_TABULEIRO, solucoes['recozimento_simulado'])
print(f"Posições das rainhas: {solucoes['recozimento_simulado']}")
print(f"score: {score(solucoes['recozimento_simulado'])}")

print('='*100)

solucoes['forca_bruta'] = brute_force(TAM_TABULEIRO, N_RAINHAS)
imprime_tabuleiro(TAM_TABULEIRO, solucoes['forca_bruta'])
print(f"Posições das rainhas: {solucoes['forca_bruta']}")
print(f"score: {score(solucoes['forca_bruta'])}")

-|-|-|-|-|-|-
-|X|-|-|-|-|-
-|-|-|-|X|-|-
-|-|-|-|-|-|-
X|-|-|-|-|-|-
-|-|-|X|-|-|-
-|-|-|-|-|X|-
Posições das rainhas: [(6, 5), (2, 4), (5, 3), (1, 1), (1, 1), (2, 4), (4, 0)]
score: 19
-|-|-|-|-|-|-
-|-|X|-|-|-|-
-|-|-|-|X|-|-
-|X|-|-|-|-|-
-|-|-|X|-|-|-
-|-|-|-|-|X|-
-|-|-|-|-|-|X
Posições das rainhas: [(6, 6), (5, 5), (4, 3), (3, 1), (3, 1), (1, 2), (2, 4)]
score: 19


 10%|█         | 8859382/85900584.0 [00:13<01:56, 660100.74it/s]

X|-|-|-|-|-|-
-|-|X|-|-|-|-
-|-|-|-|X|-|-
-|-|-|-|-|-|X
-|X|-|-|-|-|-
-|-|-|X|-|-|-
-|-|-|-|-|X|-
Posições das rainhas: [(0, 0), (1, 2), (2, 4), (3, 6), (4, 1), (5, 3), (6, 5)]
score: 21


Caso o numero de rainhas no desenho seja menor do que o fornecido, duas rainhas encontram-se na mesma posição.

É possivel notar que os métodos de simulated annealing e hill climb podem ficar presos em maximos locais, soluções não validas porem próximas de validas.